<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [102]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings
warnings.simplefilter("ignore", UserWarning) 

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 


In [104]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [105]:
# текст запроса
query_3_1 = f'''SELECT count(v.name)
                FROM public.vacancies v
                
'''

In [106]:
# результат запроса
df_vacancies = pd.read_sql_query(query_3_1, connection)
df_vacancies

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [107]:
# текст запроса
query_3_2 = f'''SELECT count(e.name)
                FROM public.employers e
                
'''

In [108]:
# результат запроса
df_employers = pd.read_sql_query(query_3_2, connection)
df_employers

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [109]:
# текст запроса
query_3_3 = f'''SELECT count(a.name)
                FROM public.areas a
                
'''

In [110]:
# результат запроса
df_areas = pd.read_sql_query(query_3_3, connection)
df_areas

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [111]:
# текст запроса
query_3_4 = f'''SELECT count(i.name)
                FROM public.industries i
                
'''

In [112]:
# результат запроса
df_industries = pd.read_sql_query(query_3_4, connection)
df_industries

,count
0,294


In [113]:
# Какие вакансии встречаются чаще всего
query_3_5=f'''SELECT distinct v.name, 
                    count(v.id)
            FROM public.vacancies v
            GROUP BY v.name
            ORDER BY 2 desc
            LIMIT 10
'''

In [114]:
number_vacancies= pd.read_sql_query(query_3_5, connection)
number_vacancies

,name,count
0,Системный администратор,1761
1,Программист 1С,1153
2,Бизнес-аналитик,488
3,Специалист технической поддержки,457
4,Системный аналитик,432
5,Аналитик,404
6,Инженер-программист,389
7,Программист 1C,360
8,Frontend-разработчик,242
9,Инженер технической поддержки,230


***

# Выводы по предварительному анализу данных
Представленных вакансий чуть больше 49 тысяч, работодателей - более 23 тысяч. Можно сказать, что в среднем один работодатель предлагает две вакансии. Отметим большое разнообразие регионов (1362) и сфер деятельности компаний (294).
В топ-10 наиболее популярных вакансий специалистов DS нет (в первой тройке системный администратор, программист 1С и бизнес-аналитик). Программист 1С появляется в десятке дважды, судя по всему, это связано с формой записи данных.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [115]:
# текст запроса
query_4_1 = f'''SELECT a.name area,
                       count(v.name) cnt
                       
                FROM public.vacancies v
                JOIN public.areas a ON a.id = v.area_id
                GROUP BY a.id
                ORDER BY count(v.name) desc
                
'''

In [116]:
# результат запроса
df_cnt_vac = pd.read_sql_query(query_4_1, connection)
df_cnt_vac

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Кизляр,1
765,Джизак,1
766,Эртиль,1
767,Арсеньев,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [117]:
# текст запроса
query_4_2 = f'''SELECT count(*)
            
               FROM public.vacancies v
               WHERE v.salary_from IS NOT NULL
                     OR v.salary_to IS NOT NULL
               
'''

In [118]:
# результат запроса
df_not_null_salary = pd.read_sql_query(query_4_2, connection)
df_not_null_salary

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [121]:
# текст запроса
query_4_3 =f'''SELECT ROUND(avg(v.salary_from),0) avg_from,
                      ROUND(avg(v.salary_to),0) avg_to
            
               FROM public.vacancies v
               WHERE v.salary_from IS NOT NULL
                     OR v.salary_to IS NOT NULL
               
'''

In [122]:
# результат запроса
df_avg_salary = pd.read_sql_query(query_4_3, connection)
df_avg_salary

,avg_from,avg_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [123]:
# текст запроса
query_4_4 = f'''SELECT v.schedule,
                      v.employment,
                      count(v.name)
                FROM public.vacancies v
                GROUP BY v.schedule, v.employment
                ORDER BY 1 desc
'''

In [124]:
# результат запроса
df_1 = pd.read_sql_query(query_4_4, connection)
df_1

,schedule,employment,count
0,Удаленная работа,Полная занятость,7802
1,Удаленная работа,Стажировка,64
2,Удаленная работа,Частичная занятость,1312
3,Удаленная работа,Проектная работа,133
4,Сменный график,Стажировка,12
5,Сменный график,Полная занятость,940
6,Сменный график,Проектная работа,1
7,Сменный график,Частичная занятость,101
8,Полный день,Проектная работа,141
9,Полный день,Стажировка,569


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [125]:
# текст запроса
query_4_5 = f'''SELECT v.experience,
                      count(v.name)
                FROM public.vacancies v
                GROUP BY v.experience
                ORDER BY 2
'''

In [126]:
# результат запроса
df_experience = pd.read_sql_query(query_4_5, connection)
df_experience

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

# Выводы по детальному анализу вакансий
Зарплата указана у менее половины вакансий. Наибольшее число вакансий предполагают полный день/полную занятость, хотя кажется, что IT вакансии во многом удаленная работа.  Наибольшее количество предложенных вакансий находятся в городах миллионниках. Вариантов стажировок и проектной работы среди вакансий относительно немного. Так же следует отметить, что почти половина вакансий требует достаточно небольшой опыт работы, от 1 до 3 лет.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [129]:
# текст запроса
query_5_1 = f'''(SELECT e.name,
                        count(v.name)
                      
               FROM public.vacancies v
               JOIN public.employers e ON e.id=v.employer_id 
               GROUP BY e.name
               ORDER BY 2 desc
               LIMIT  1)
               
               UNION ALL
               
               (SELECT e.name,
                       count(v.name)
                      
               FROM public.vacancies v
               JOIN public.employers e ON e.id=v.employer_id 
               GROUP BY e.name
               ORDER BY 2 desc
               OFFSET 4 
               LIMIT  1)
               
'''

In [130]:
# результат запроса
df_employers_vac = pd.read_sql_query(query_5_1, connection)
df_employers_vac

,name,count
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [131]:
# текст запроса
query_5_2 = f'''SELECT a.name area,
                      count(distinct e.name) employers,
                      count(distinct v.name) vacancion
               FROM public.areas a
               LEFT JOIN public.vacancies v ON a.id=v.area_id
               LEFT JOIN public.employers e ON a.id=e.area
               WHERE v.id IS NULL
               GROUP BY a.id
               ORDER BY 2 desc
               LIMIT  1
'''

In [132]:
# результат запроса
df_area_emp_vac = pd.read_sql_query(query_5_2, connection)
df_area_emp_vac

,area,employers,vacancion
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [133]:
# текст запроса
query_5_3 = f'''SELECT e.name,
                      count(distinct a.name)
            FROM public.vacancies v
            LEFT JOIN public.employers e ON v.employer_id=e.id
            LEFT JOIN public.areas a ON v.area_id=a.id
            GROUP BY e.name
            ORDER BY 2 desc
'''

In [134]:
# результат запроса
df_emp_area_vac = pd.read_sql_query(query_5_3, connection)
df_emp_area_vac

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [135]:
# текст запроса
query_5_4 =f'''SELECT count(e.id) employers_count
              FROM public.employers e
              LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id
              
              WHERE ei.employer_id IS NULL
'''

In [136]:
# результат запроса
df_emp=pd.read_sql_query(query_5_4, connection)
df_emp

,employers_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [137]:
# текст запроса
query_5_5 = f'''SELECT e.name

               FROM public.employers e
               LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id
               
               GROUP BY e.name
               HAVING count(ei.industry_id)=4
               ORDER BY 1
               OFFSET 2 
               LIMIT 1
'''

In [138]:
# результат запроса
df_emp_ind = pd.read_sql_query(query_5_5, connection)
df_emp_ind

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [139]:
# текст запроса
query_5_6 = f'''SELECT count(e.id)
                      
               FROM public.employers e
               LEFT JOIN public.employers_industries ei ON ei.employer_id=e.id
               LEFT JOIN public.industries i ON ei.industry_id=i.id
               WHERE i.name='Разработка программного обеспечения'

'''

In [140]:
# результат запроса
df_empl_po = pd.read_sql_query(query_5_6, connection)
df_empl_po

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [141]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url)
table = pd.read_html(url, match='Город')[0]
cities_list = list(table['Город'])
cities_list = tuple([w for w in cities_list if w.strip()])
print(cities_list)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [142]:
# текст запроса
query_5_7 = f'''(SELECT a.name area,
                      count(v.id)
                      
               FROM public.vacancies v
               JOIN public.areas a ON a.id = v.area_id
               JOIN public.employers e ON e.id = v.employer_id
               WHERE e.name='Яндекс' 
                     AND a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград') 
               GROUP BY a.name
               )
               
               UNION ALL
               
               (SELECT 'Total',
                       count(v.id) 
                      
               FROM public.vacancies v
               JOIN public.areas a ON a.id = v.area_id
               JOIN public.employers e ON e.id = v.employer_id
               WHERE e.name='Яндекс' 
                     AND a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Краснодар', 'Самара', 'Ростов-на-Дону', 'Омск', 'Воронеж', 'Пермь', 'Волгоград'))
               ORDER BY 2

'''

In [143]:
# результат запроса
yandex_areas = pd.read_sql_query(query_5_7, connection)
yandex_areas

,area,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

# Выводы по анализу работодателей
Лидер по колличеству вакансий  - это "Яндекс": он предлагает больше всего вакансий в самом большом числе регионов. Также в топе находятся другие широко известные компании: Ростелеком, Тинькофф, Сбер, Газпром и так далее.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [144]:
# текст запроса
query_6_1 = f'''SELECT count(v.name)
                      
            FROM public.vacancies v
            
            WHERE lower(v.name) like '%data%' 
                  OR lower(v.name) like '%данн%'
'''

In [145]:
# результат запроса
data_vacancies = pd.read_sql_query(query_6_1, connection)
data_vacancies

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [146]:
# текст запроса
query_6_2 = f'''SELECT count(*) cnt
                      
            FROM public.vacancies v
            
            WHERE 
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR ((lower(v.name) LIKE lower('%ML%')) and (lower(v.name) NOT LIKE lower('%HTML%')))
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
                AND ((lower(v.name) LIKE '%junior%')
                 OR (v.experience ='Нет опыта') 
                 OR (v.employment ='Стажировка'))
'''


In [147]:
# результат запроса
junior_vacancies = pd.read_sql_query(query_6_2, connection)
junior_vacancies

,cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [148]:
# текст запроса
query_6_3 = f'''select count(*) cnt
                      
            from public.vacancies v
            
           WHERE 
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR ((lower(v.name) LIKE lower('%ML%')) and (lower(v.name) NOT LIKE lower('%HTML%')))
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
                
               AND (lower(v.key_skills) LIKE '%postgres%' 
               OR lower(v.key_skills) LIKE '%sql%')
                   
'''

In [149]:
# результат запроса
sql_postgres_skills = pd.read_sql_query(query_6_3, connection)
sql_postgres_skills

,cnt
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''SELECT count(v.name) cnt
                      
            FROM public.vacancies v
            
            WHERE 
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR ((lower(v.name) LIKE lower('%ML%')) AND (lower(v.name) NOT LIKE lower('%HTML%')))
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
            AND lower(v.key_skills) LIKE lower('%Python%') 
'''

In [151]:
# результат запроса
python_skills = pd.read_sql_query(query_6_4, connection)
python_skills

,cnt
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [157]:
# текст запроса
# Ключевые навыки в каждой строке разделены символами табуляции, поэтому мы заменим этот символ пустой строкой (по сути удалим)
# Далее вычтем из полной строки получившуются сокращенную строку и прибавим 1 (так как один символ разделяет два навыка)
query_6_5 = f'''SELECT
                       round(avg(1+(length(v.key_skills) - length(coalesce(replace(v.key_skills, CHR(9), ''))))), 2)
            FROM public.vacancies v
            WHERE
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') 
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%') 
            AND (v.key_skills) IS NOT NULL
           
                
'''

In [158]:
# результат запроса
skills=pd.read_sql_query(query_6_5, connection)
skills

,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [159]:
# текст запроса
query_6_6 = f'''SELECT  
                        v.experience,
                       ROUND(AVG(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0)
                       
            FROM public.vacancies v
            WHERE 
                v.experience='От 3 до 6 лет' 
                AND
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR (v.name LIKE '%ML%' AND v.name NOT LIKE '%HTML%') 
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%') 
                AND
                (v.salary_from IS NOT NULL or v.salary_to IS NOT NULL)
            GROUP BY v.experience
                
'''

In [160]:
# результат запроса
avg_salary=pd.read_sql_query(query_6_6, connection)
avg_salary

,experience,round
0,От 3 до 6 лет,243115.0


***

# Выводы по предметному анализу
Всего вакансий, которыйе относятся с работой с данными - 1771, эот 3,5% от общего числа вакансий. При этом вакансий для начинающего дата-сайентиста всего 51. В зарплатах мы наблюдаем наибольший разрыв между неопытными сотрудниками и теми, чей опыт - от 1 до 3 лет. Имея средний опыт, можно получать почти вдвое большую зарплату, чем на начальном уровне. Зарплатный разрыв между сотрудниками со средним и большим (более 3 лет) опытом также велик: около 140к и примерно 243к. 
Ключевых навыков, которые ожидают работодатели, в среднем 6, причем весьма популярен Python - его требуют в чуть более 20% вакансий. SQL менее популярен, его ожидают примерно в 13% вакансий.

# Общий вывод по проекту

In [161]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
# В качестве дополнительного исследования давайте посмотрим, в каких городах есть вакансии DS
query_7_1 = f'''SELECT 
                       a.name area,
                       count(v.id) cnt
            FROM public.areas a
            JOIN public.vacancies v ON a.id=v.area_id
            
            WHERE 
                (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR ((v.name ILIKE '%ML%') AND (v.name NOT ILIKE '%HTML%'))
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
            
            GROUP BY a.name 
            ORDER BY cnt desc
'''

In [162]:
# результат запроса
ds_area=pd.read_sql_query(query_7_1, connection)
ds_area

,area,cnt
0,Москва,225
1,Санкт-Петербург,69
2,Новосибирск,23
3,Алматы,22
4,Нижний Новгород,20
5,Казань,19
6,Томск,11
7,Ростов-на-Дону,11
8,Минск,11
9,Пермь,8


In [163]:
query_7_2 = f'''SELECT count(v.id) cnt,
                       v.schedule
                      
            FROM public.vacancies v 
            
            WHERE 
                v.schedule = 'Удаленная работа'
                AND (lower(v.name) LIKE '%data scientist%' 
                OR lower(v.name) LIKE '%data science%'
                OR lower(v.name) LIKE '%исследователь данных%' 
                OR ((v.name ILIKE '%ML%') AND (v.name not ILIKE '%HTML%'))
                OR lower(v.name) LIKE '%machine learning%'
                OR lower(v.name) LIKE '%машинн%обучен%')
            
            GROUP BY  v.schedule
            ORDER BY cnt desc
'''

In [164]:
# результат запроса
ds_schedule=pd.read_sql_query(query_7_2, connection)
ds_schedule

,cnt,schedule
0,129,Удаленная работа


Одназначным лидером по количеству DS вакансий является Москва, с огромным отрывом от нее идут Санкт-Петербург, Новосибирск, Нижний Новогород и другие города России и ближнего зарубежья.
В целом шансы найти работу в области DS значительно зависят от региона проживания соискателя (или его готовности переехать).

специальность 'Data science' однозначно не является наиболее популярной среди других it-специальностей в данной базе данных, можно сказать, что работа в этой области есть как для начинающих, так и для опытных дата сайентистов. При этом наличие хотя бы одного года работы значительно увеличивает число подходящих вакансий. 
Распределение вакансий по регионам предсказуемо: лучше всего искать работу в Москве. Варианты удаленной работы составляют 7.2% от общего числа вакансий связанных с данными. 

In [165]:
# не забываем закрыть соединение после окончания работы
connection.close()